### Instalação do XGBoost

1. Vá em File -> New -> Terminal e digite o seguinte comando:

> conda install -c conda-forge xgboost


2. Depois, execute o código abaixo para ver se a instalação funcionou corretamente

In [1]:
import xgboost

### Versões dos pacotes

1. Confira também se os pacotes abaixo estão nas seguintes versões:

> Python >= 3.7

> numpy >= 1.19.1

> SciPy >= 1.5.2

> Scikit-Learn >= 0.23.2

> XGBoost >= 1.2.0

In [2]:
import platform; print(platform.platform())
import sys; print("Python", sys.version)
import numpy; print("NumPy", numpy.__version__)
import scipy; print("SciPy", scipy.__version__)
import sklearn; print("Scikit-Learn", sklearn.__version__)
import xgboost; print("XGBoost", xgboost.__version__)

macOS-10.16-x86_64-i386-64bit
Python 3.10.9 (main, Mar  1 2023, 12:33:47) [Clang 14.0.6 ]
NumPy 1.23.5
SciPy 1.10.0
Scikit-Learn 1.2.1
XGBoost 1.7.4


# Criando algoritmos baselines

Antes de analisarmos o algoritmo de Árvore de Decisão, vamos criar aqui algoritmos baseline e compara-los minimanente com o XGBoost, tanto para regressão quanto para classificação. Isso será importante para podermos comparar com os resultados de quando fizermos os ajustes nos hiperparâmetros do XGBoost

### Regressão

In [3]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [4]:
df_bikes = pd.read_csv('bases/bike_rentals_cleaned.csv', sep=',')
df_bikes.head()

,instant,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,cnt
0,1,1.0,0.0,1,0.0,6.0,0.0,2,0.344167,0.363625,0.805833,0.160446,985
1,2,1.0,0.0,1,0.0,0.0,0.0,2,0.363478,0.353739,0.696087,0.248539,801
2,3,1.0,0.0,1,0.0,1.0,1.0,1,0.196364,0.189405,0.437273,0.248309,1349
3,4,1.0,0.0,1,0.0,2.0,1.0,1,0.200000,0.212122,0.590435,0.160296,1562
4,5,1.0,0.0,1,0.0,3.0,1.0,1,0.226957,0.229270,0.436957,0.186900,1600


In [5]:
#separando os dados em features e labels
X = df_bikes.iloc[:,:-1]
y = df_bikes.iloc[:,-1]

In [6]:
#vamos treinar uma regressão linear e comparar com o XGBoost depois
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=2)

In [7]:
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)

y_pred = lin_reg.predict(X_test)

from sklearn.metrics import mean_squared_error
import numpy as np

mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

print("RMSE: %0.2f" % (rmse))

RMSE: 898.21


In [8]:
#podemos usar o método describe para tentar qualificar o RMSE
df_bikes['cnt'].describe()

count     731.000000
mean     4504.348837
std      1937.211452
min        22.000000
25%      3152.000000
50%      4548.000000
75%      5956.000000
max      8714.000000
Name: cnt, dtype: float64

In [9]:
from xgboost import XGBRegressor
xg_reg = XGBRegressor()

xg_reg.fit(X_train, y_train)
y_pred = xg_reg.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

print("RMSE: %0.2f" % (rmse))

RMSE: 705.11


### Cross-Validation

Realizar apenas um teste não é confiável, visto que se você fizer uma divisão dos dados diferente, você obterá diferentes resultados. Para tratar isso, usamos cross-validation, cujo processo já foi explicado na aula de SVM.

In [10]:
#para regressão linear
from sklearn.model_selection import cross_val_score
model = LinearRegression()
scores = cross_val_score(model, X, y, scoring='neg_mean_squared_error', cv=10)
rmse = np.sqrt(-scores)
print('Reg rmse:', np.round(rmse, 2))
print('RMSE mean: %0.2f' % (rmse.mean()))

Reg rmse: [ 504.01  840.55 1140.88  728.39  640.2   969.95 1133.45 1252.85 1084.64
 1425.33]
RMSE mean: 972.02


In [11]:
# para XGBoost
model = XGBRegressor()
scores = cross_val_score(model, X, y, scoring='neg_mean_squared_error', cv=10)
rmse = np.sqrt(-scores)
print('Reg rmse:', np.round(rmse, 2))
print('RMSE mean: %0.2f' % (rmse.mean()))

Reg rmse: [ 717.65  692.8   520.7   737.68  835.96 1006.24  991.34  747.61  891.99
 1731.13]
RMSE mean: 887.31


### Classificação

In [12]:
df_census = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data', header=None)
df_census.columns = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation',
                  'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 
                   'income']
df_census.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [13]:
df_census.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  int64 
 1   workclass       32561 non-null  object
 2   fnlwgt          32561 non-null  int64 
 3   education       32561 non-null  object
 4   education-num   32561 non-null  int64 
 5   marital-status  32561 non-null  object
 6   occupation      32561 non-null  object
 7   relationship    32561 non-null  object
 8   race            32561 non-null  object
 9   sex             32561 non-null  object
 10  capital-gain    32561 non-null  int64 
 11  capital-loss    32561 non-null  int64 
 12  hours-per-week  32561 non-null  int64 
 13  native-country  32561 non-null  object
 14  income          32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


Quando olhamos para o dataset, vemos a coluna 'education' e 'education_num'. A última é a versão numérica da primeira. Assim, podemos dropar 'education'. 

In [14]:
df_census = df_census.drop(['education'], axis=1)

Todas as colunas precisam ser transformadas em colunas numéricas. O método *get_dumies* do pandas pega valores únicos não-numéricos de cada coluna e os converte em colunas, com 1 indicando presença e 0 ausência. Veja o exemplo 

In [15]:
#exemplo
pd.get_dummies(df_census['marital-status'])

,Divorced,Married-AF-spouse,Married-civ-spouse,Married-spouse-absent,Never-married,Separated,Widowed
0,0,0,0,0,1,0,0
1,0,0,1,0,0,0,0
2,1,0,0,0,0,0,0
3,0,0,1,0,0,0,0
4,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...
32556,0,0,1,0,0,0,0
32557,0,0,1,0,0,0,0
32558,0,0,0,0,0,0,1
32559,0,0,0,0,1,0,0


In [16]:
df_census = pd.get_dummies(df_census)
df_census.head()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,workclass_ ?,workclass_ Federal-gov,workclass_ Local-gov,workclass_ Never-worked,...,native-country_ Scotland,native-country_ South,native-country_ Taiwan,native-country_ Thailand,native-country_ Trinadad&Tobago,native-country_ United-States,native-country_ Vietnam,native-country_ Yugoslavia,income_ <=50K,income_ >50K
0,39,77516,13,2174,0,40,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
1,50,83311,13,0,0,13,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
2,38,215646,9,0,0,40,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
3,53,234721,7,0,0,40,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
4,28,338409,13,0,0,40,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


O objetivo (coluna target) é determinar se uma pessoa ganha mais que 50k ou não. Depois do pd.get_dummies, duas colunas, df_census['income_<=50k'] e df_census['income_>50k] são usadas para determinar se uma pessoa ganha 50k. Visto que ambas representam a mesma coisa, vamos deletar a primeira

In [17]:
df_census = df_census.drop('income_ <=50K', axis=1)

In [18]:
#features e labels
X = df_census.iloc[:,:-1]
y = df_census.iloc[:,-1]

In [19]:
from sklearn.neighbors import KNeighborsClassifier

In [20]:
#uma função para executar cross-validation
def cross_val(classifier, num_splits=10):
    model = classifier
    scores = cross_val_score(model, X, y, cv=num_splits)
    print('Accuracy:', np.round(scores, 2))
    print('Accuracy mean: %0.2f' % (scores.mean()))

In [21]:
cross_val(KNeighborsClassifier())

Accuracy: [0.77 0.78 0.78 0.78 0.78 0.78 0.77 0.78 0.77 0.78]
Accuracy mean: 0.78


In [22]:
from xgboost import XGBClassifier

In [23]:
cross_val(XGBClassifier(n_estimators=5))

Accuracy: [0.85 0.86 0.87 0.85 0.86 0.86 0.86 0.87 0.86 0.86]
Accuracy mean: 0.86


# Decision Trees

Vamos começar executando decision trees para comparar os resultados com o dataset df_census e depois entender como usar GridSearch para encontrar os melhores parâmetros para nossa árvore de decisão

In [24]:
X = df_census.iloc[:,:-1]
y = df_census.iloc[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=2)

In [25]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
clf = DecisionTreeClassifier(random_state=2)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
accuracy_score(y_pred, y_test)

0.8131679154894976

Agora vamos treinar uma árvore de regressão usando GridSearch

In [26]:
df_bikes = pd.read_csv('bases/bike_rentals_cleaned.csv')

X_bikes = df_bikes.iloc[:,:-1]
y_bikes = df_bikes.iloc[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X_bikes, y_bikes, random_state=2)

In [27]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score

In [28]:
reg = DecisionTreeRegressor(random_state=2)
scores = cross_val_score(reg, X_bikes, y_bikes, scoring='neg_mean_squared_error', cv=5)
rmse = np.sqrt(-scores)
print('RMSE mean: %0.2f' % (rmse.mean()))

RMSE mean: 1233.36


Podemos checar se o modelo esta overfitando ou não. para isso, podemos predizer o proprio conjunto de treino e verificar o RMSE

In [29]:
reg = DecisionTreeRegressor()
reg.fit(X_train, y_train)
y_pred = reg.predict(X_train)
from sklearn.metrics import mean_squared_error
reg_mse = mean_squared_error(y_train, y_pred)
reg_rmse = np.sqrt(reg_mse)
reg_rmse

0.0

Como suspeitamos, o modelo está overfitando. Assim, precisamos usar o GridSearchCV para determinar o melhor valor dos hiperparâmetos.

Vamos começar por *max_depth*

In [30]:
from sklearn.model_selection import GridSearchCV
params = {'max_depth':[None,2,3,4,6,8,10,20]}
reg = DecisionTreeRegressor(random_state=2)
grid_reg = GridSearchCV(reg, params, scoring='neg_mean_squared_error', cv=5, n_jobs=-1)


grid_reg.fit(X_train, y_train)

best_params = grid_reg.best_params_
print("Best params:", best_params)

Best params: {'max_depth': 6}


In [31]:
#e agora verificamos se com esse valor, o RMSE diminui
best_model = grid_reg.best_estimator_
y_pred = best_model.predict(X_test)
rmse_test = mean_squared_error(y_test, y_pred)**0.5

print('Test score: {:.3f}'.format(rmse_test))

Test score: 864.670


Para testar com outras features, vamos criar uma função para facilitar nosso trabalho:

In [32]:
def grid_search(params, reg=DecisionTreeRegressor(random_state=2)):

    grid_reg = GridSearchCV(reg, params, scoring='neg_mean_squared_error', cv=5, n_jobs=-1)
    
    grid_reg.fit(X_train, y_train)

    best_params = grid_reg.best_params_

    print("Best params:", best_params)
    
    best_score = np.sqrt(-grid_reg.best_score_)

    print("Training score: {:.3f}".format(best_score))

    y_pred = grid_reg.predict(X_test)

    rmse_test = mean_squared_error(y_test, y_pred)**0.5

    print('Test score: {:.3f}'.format(rmse_test))

In [33]:
#vamos testar para a feature min_samples_leaf
grid_search(params={'min_samples_leaf':[1,2,4,6,8,10,20,30]})

Best params: {'min_samples_leaf': 8}
Training score: 896.083
Test score: 855.620


Vamos ver o que acontece quando usamos as duas features no gridsearch

In [34]:
grid_search(params={'max_depth':[None,2,3,4,6,8,10,20],'min_samples_leaf':[1,2,4,6,8,10,20,30]})

Best params: {'max_depth': 6, 'min_samples_leaf': 2}
Training score: 870.396
Test score: 913.000


Surpreendentemente, o score aumentou. Precisamos sempre colocar vários parÂmetros juntos para executar o GridSearch.

vamos iniciar min_samples_leaf em 3 e max_depth a partir de 6 e ver o que acontece:

In [35]:
grid_search(params={'max_depth':[5,6,7,8,9],'min_samples_leaf':[3,5,7,9]})

Best params: {'max_depth': 9, 'min_samples_leaf': 7}
Training score: 888.905
Test score: 878.538


Vimos que o score melhorou, mas ainda temos um longo caminho pela frente. 

# Bagging

### Random Forest Classifiers

Vamos começar usando o dataset df_census e comparar o Random Forest com Árvores de Decisão e XGBoost

In [36]:
from sklearn.ensemble import RandomForestClassifier

In [37]:
X_census = df_census.iloc[:,:-1]
y_census = df_census.iloc[:,-1]

In [38]:
rf = RandomForestClassifier(n_estimators=10, random_state=2, n_jobs=-1)

#n_estimators = número de árvores
#n_jobs = quando igual a -1, usa todos os processadores para execução em paralelo

scores = cross_val_score(rf, X_census, y_census, cv=5)
print('Accuracy:', np.round(scores, 3))
print('Accuracy mean: %0.3f' % (scores.mean()))

Accuracy: [0.851 0.844 0.851 0.852 0.851]
Accuracy mean: 0.850


A versão default do Random Forest forneceu um resultado melhor que o das Árvores de Decisão (81%), mas não tão bom quanto o XGBoost (86%). 

Mas aqui, a grande questão é: porque RF foi melhor que uma árvore de decisão individual?

A resposta curta é: **bagging**. Com 10 árvores (n_estimator=10), cada predição é baseada em 10 árvores de decisão ao invés de em apenas uma. O Bootstrap age também, fornecendo diversidade e reduzindo a variância.

Por padrão, Random Forest Classifiers selecionam a partir da raiz quadrada do número total de features ao fazer um split. Então, se existem 100 features (Colunas), cada árvore de decisão vai considerar apenas 10 features quando fazer o split. Assim, duas árvores com amostras duplicadas podem fornecer predições muito diferentes devido aos diferentes splits. Essa é outra maneira de reduzir variância.  

### Random Forest Regressor

Numa Random Forest Regressor, os exemplos são amostrados com reposição, da mesma maneira que num Random Forest Classifier, mas o número máximo de features é o número total de features ao invés da raiz quadrada. Isto é devido a experimentos. Veja [aqui](https://orbi.uliege.be/bitstream/2268/9357/1/geurts-mlj-advance.pdf)

Além disso, a predição final é feita levando em consideração a média das predições de todas as árvores ao invés de regras baseadas em voto majoritário. 

In [39]:
X_bikes = df_bikes.iloc[:,:-1]
y_bikes = df_bikes.iloc[:,-1]

In [40]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators=10, random_state=2, n_jobs=-1)
scores = cross_val_score(rf, X_bikes, y_bikes, scoring='neg_mean_squared_error', cv=10)

rmse = np.sqrt(-scores)
print('RMSE:', np.round(rmse, 3))
print('RMSE mean: %0.3f' % (rmse.mean()))

RMSE: [ 801.486  579.987  551.347  846.698  895.05  1097.522  893.738  809.284
  833.488 2145.046]
RMSE mean: 945.365


Vamos agora ajustar os hiperparâmetros do Random Forest e aproveitar para introduzir uma nova maneira de fazer GridSearch, chamada de RandomizedSearchCV. A ideia é a mesma por trás do GridSearchCV, porém, como este pode levar muito tempo, RandomizedSearchCV trabalha escolhendo um número aleatório de combinações. Não foi feito para ser exaustivo, mas sim para encontrar a melhor combinação de parâmetros num tempo limitado.


Vamos voltar a dataset df_bikes

In [41]:
X_train, X_test, y_train, y_test = train_test_split(X_bikes, y_bikes, random_state=2)

In [42]:
rf = RandomForestRegressor(n_estimators=50, warm_start=True, n_jobs=-1, random_state=2)
scores = cross_val_score(rf, X_bikes, y_bikes, scoring='neg_mean_squared_error', cv=10)
rmse = np.sqrt(-scores)
print('RMSE:', np.round(rmse, 3))
print('RMSE mean: %0.3f' % (rmse.mean()))

RMSE: [ 836.482  541.898  533.086  812.782  894.877  881.117  794.103  828.968
  772.517 2128.148]
RMSE mean: 902.398


Introduzimos um novo parâmetro aqui: **warm_start**.

Quando warm_star=True, adicionar mais árvores não requer começar tudo do zero. Se você mudar n_stimators de 100 para 200, Random Forest vai reusar a solução do fit() anterior, tornando a execução mais rápida. 

Agora vamos usar o RandomizedSearchCV. Para isso, vamos criar uma função que recebe os parâmetros que eu quero otimizar, a quantidade de vezes que vamos executar o algoritmo e o regressor.

In [43]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error as MSE

def randomized_search_reg(params, runs=16, reg=RandomForestRegressor(random_state=2, n_jobs=-1)):
    rand_reg = RandomizedSearchCV(reg, params, n_iter=runs, scoring='neg_mean_squared_error', 
                                  cv=10, n_jobs=-1, random_state=2)
    
    rand_reg.fit(X_train, y_train)

    best_model = rand_reg.best_estimator_

    best_params = rand_reg.best_params_

    print("Best params:", best_params)
    
    best_score = np.sqrt(-rand_reg.best_score_)

    print("Training score: {:.3f}".format(best_score))

    y_pred = best_model.predict(X_test)

    rmse_test = MSE(y_test, y_pred)**0.5
    
    print('Test set score: {:.3f}'.format(rmse_test))

In [44]:
#agora executamos

randomized_search_reg(params={
                          'min_samples_leaf':[1,2,4,6,8,10,20,30],
                          'min_impurity_decrease':[0.0, 0.01, 0.05, 0.10, 0.15, 0.2],
                          'max_features':['auto', 0.8, 0.7, 0.6, 0.5, 0.4],
                          'max_depth':[None,4,6,8,10,12,15,20]
                         }, runs=20)

/Users/dhenyfernandes/anaconda3/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/Users/dhenyfernandes/anaconda3/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/Users/dhenyfernandes/anaconda3/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this param

Best params: {'min_samples_leaf': 1, 'min_impurity_decrease': 0.1, 'max_features': 0.6, 'max_depth': 12}
Training score: 675.128
Test set score: 619.014


# Boosting

Para entendermos a maneira correra que o Gradient Boosting funciona, vamos usar uma Árvore de Regressão. 

In [45]:
X_bikes = df_bikes.iloc[:,:-1]
y_bikes = df_bikes.iloc[:,-1]

X_train, X_test, y_train, y_test = train_test_split(X_bikes, y_bikes, random_state=2)

1. Aqui, vamos usar um base learner, então configuramos max_depth=2, pois queremos aprender a partir dos erros.

In [46]:
# Vamos treinar um Decision tree Regressor
tree_1 = DecisionTreeRegressor(max_depth=2, random_state=2)
tree_1.fit(X_train, y_train)

DecisionTreeRegressor(max_depth=2, random_state=2)

2. Vamos fazer predição com o conjunto de treino: ao invés de fazer predições com o conjunto de teste, no gradient boosting as predições são feitas inicialmente com o conjunto de treino. Porque? Para comparar os residuals, precisamos comparar as predições enquanto estamos na fase de treino. A fase de teste do modelo acontece no final, quando todas as árvores estiverem construídas. 

In [47]:
y_train_pred = tree_1.predict(X_train)

3. Agora, calculamos os residuals: a diferença entre o predito e o observado

In [48]:
y2_train = y_train - y_train_pred

**NOTA**: os residuals são definidos como y2_train por que agora eles são as labels da próxima árvore

4. Treine uma nova árvore em cima dos residuals. Isso vai, gradativamente, diminuindo os residuals

In [49]:
tree_2 = DecisionTreeRegressor(max_depth=2, random_state=2)
tree_2.fit(X_train, y2_train)

DecisionTreeRegressor(max_depth=2, random_state=2)

5. Repita os passos 2 a 4. Conforme o processo continua, os residuals gradativamente vão se aproximando de 0. 

In [50]:
y2_train_pred = tree_2.predict(X_train)
y3_train = y2_train - y2_train_pred


tree_3 = DecisionTreeRegressor(max_depth=2, random_state=2)
tree_3.fit(X_train, y3_train)

DecisionTreeRegressor(max_depth=2, random_state=2)

6. Agora somamos os resultados: somar os resultados requer fazer predições para cada árvore com o conjunto de teste.

In [51]:
y1_pred = tree_1.predict(X_test)

y2_pred = tree_2.predict(X_test)

y3_pred = tree_3.predict(X_test)

y_pred = y1_pred + y2_pred + y3_pred

7. Agora, podemos calcular o MSE para obter os resultados

In [52]:
MSE(y_test, y_pred)**0.5
#um resultado excelente para um weak learner, principalmente quando comparamos com o resultado da 
#árvore de regressão

911.0479538776444

### Gradient Boosting usando sklearn

Podemos treinar um Gradiet Boosting usando a scikit-learn para ver se chegamos no mesmo resultado

In [53]:
from sklearn.ensemble import GradientBoostingRegressor

In [54]:
gbr = GradientBoostingRegressor(max_depth=2, n_estimators=3, random_state=2, learning_rate=1.0)

gbr.fit(X_train, y_train)
y_pred = gbr.predict(X_test)
MSE(y_test, y_pred)**0.5


911.0479538776439

Temos um novo parâmetro: learning_rate. Para entender sua influência (e também no XGBoost), vamos treinar dois GBs mudando somente *n_estimators*:

In [55]:
gbr = GradientBoostingRegressor(max_depth=2, n_estimators=30, random_state=2, learning_rate=1.0)
gbr.fit(X_train, y_train)
y_pred = gbr.predict(X_test)
MSE(y_test, y_pred)**0.5

857.1072323426944

Uma melhora significativa. Vamos testar com 300, então.

In [56]:
gbr = GradientBoostingRegressor(max_depth=2, n_estimators=300, random_state=2, learning_rate=1.0)
gbr.fit(X_train, y_train)
y_pred = gbr.predict(X_test)
MSE(y_test, y_pred)**0.5

936.3617413678853

Opa! Algo de errado não está certo. Aqui vemos a importância do learning_rate. 

Basicamente, a ideia consiste em reduzir a contribuição de árvores individuais de modo que nenhuma árvore tenha muita influência ao construir o modelo, ou seja, o learning rate limita a influência de árvores individuais. Assim, se diminuirmos seu valor, veremos uma grande diferença:

In [57]:
gbr = GradientBoostingRegressor(max_depth=2, n_estimators=300, random_state=2, learning_rate=0.1) #default sklearn
gbr.fit(X_train, y_train)
y_pred = gbr.predict(X_test)
MSE(y_test, y_pred)**0.5

653.7456840231495

Agora, podemos executar um RandomizedSearchCV e encontrar os melhores parâmetros (Dentro de um range específico)

In [58]:
params={'subsample':[0.65, 0.7, 0.75],
                          'n_estimators':[300, 500, 1000],
                          'learning_rate':[0.05, 0.075, 0.1]
       }
gbr = GradientBoostingRegressor(max_depth=3, random_state=2)

rand_reg = RandomizedSearchCV(gbr, params, n_iter=10, scoring='neg_mean_squared_error', 
                              cv=5, n_jobs=-1, random_state=2)

rand_reg.fit(X_train, y_train)


best_model = rand_reg.best_estimator_
best_params = rand_reg.best_params_
print("Best params:", best_params)
best_score = np.sqrt(-rand_reg.best_score_)
print("Training score: {:.3f}".format(best_score))
y_pred = best_model.predict(X_test)
rmse_test = MSE(y_test, y_pred)**0.5
print('Test set score: {:.3f}'.format(rmse_test))

Best params: {'subsample': 0.65, 'n_estimators': 300, 'learning_rate': 0.05}
Training score: 636.200
Test set score: 625.985


**NOTA**: usamos um parâmetro a mais: subsample. Como o nome sugere, é um subconjunto de amostras. Isso signifca que nem todas as linhas serão incluídas ao construir uma árvore. Quando seu valor é diferente de 1.0, o modelo é classificado como **Stochastic Gradient Descent**, em que *stochastic* significa alguma aleatoriedade é inerente ao modelo. 

# XGBoost

## Classification

In [59]:
from sklearn import datasets
iris = datasets.load_iris()

In [60]:
df = pd.DataFrame(data= np.c_[iris['data'], iris['target']],columns= iris['feature_names'] + ['target'])
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0


In [61]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

In [62]:
X_train, X_test, y_train, y_test = train_test_split(iris['data'], iris['target'], random_state=2)

In [63]:
xgb = XGBClassifier(booster='gbtree', objective='multi:softprob', 
                    learning_rate=0.1, n_estimators=100, random_state=2, n_jobs=-1)
xgb.fit(X_train, y_train)
y_pred = xgb.predict(X_test)
score = accuracy_score(y_pred, y_test)
print('Score: ' + str(score))

Score: 0.9736842105263158


Algumas considerações sobre os parâmetros:

1. **booster='gbtree'**: booster é o *base learner*. 'gbtree' é o gradient boosted tree

2. **objective='multi:softprob'**: função objetivo. No caso da classificação, calcula a probabilidade de cada classe e escolhe aquela com maior valor. 

## Regression

In [64]:
from xgboost import XGBRegressor

X,y = datasets.load_diabetes(return_X_y=True)

xgb = XGBRegressor(booster='gbtree', objective='reg:squarederror', 
                    learning_rate=0.1, n_estimators=100, random_state=2, n_jobs=-1)

scores = cross_val_score(xgb, X, y, scoring='neg_mean_squared_error', cv=5)
rmse = np.sqrt(-scores)
print('RMSE:', np.round(rmse, 3))
print('RMSE mean: %0.3f' % (rmse.mean()))

RMSE: [63.011 59.705 64.538 63.706 64.588]
RMSE mean: 63.109


1. **objective='reg:squarederror'**: é o MSE a partir do qual derivamos a função objetivo

In [65]:
#para poder comparar o resultado, podemos verificar usar o método describe na variável y (label) e obter 
#suas principais estatísticas

In [66]:
pd.DataFrame(y).describe()

,0
count,442.000000
mean,152.133484
std,77.093005
min,25.000000
25%,87.000000
50%,140.500000
75%,211.500000
max,346.000000


O resultado de 63.124 é menor que 1 desvio padrão, o que é um resultado respeitável. 

# Conclusão

Agora, temos um template tanto para classificação quanto para regressão que podemos usar como ponto de partida para resolver problemas usando XGBoost.